In [215]:
import pandas as pd
import plotly.graph_objs as go


In [216]:
data = pd.read_csv('data/GGGI/GGIs_2015_2020.csv')

121

In [217]:
# Create "avg countries"

In [218]:
Income_region_group = data.groupby(['Variable', 'Year', 'IncomeLevel', 'Region', 'Aggregation']).mean().reset_index()
Income_region_group['ISO'] = 'AVG' + '_' + Income_region_group["IncomeLevel"] + '_' + Income_region_group["Region"] 

In [219]:
data = pd.concat([data, Income_region_group])

# Trends

In [231]:
ISO = 'ITA'

REF = 'AVG_' + "_".join(data[data.ISO == ISO][["IncomeLevel", 'Region']].drop_duplicates().values[0].tolist())

df = data[(data.ISO.isin([ISO, REF])) & (data.Aggregation == 'Index')]

fig = px.line(df,
              x='Year',
              y='Value',
              color='ISO',
              line_dash='ISO',
              color_discrete_map={ISO: '#14ac9c', REF:'darkgrey'},
              height=300)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(visible=True, fixedrange=True)
fig.update_traces(mode='lines+markers')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

# World outlouk

## Dotplot

In [321]:
df = data[(data.Aggregation.isin(['Dimension'])) & (data.Year.isin([2015, 2020]))].groupby(['Variable', 'Year']).mean().reset_index()
df['Value'] = df['Value'].round(1)
df['Year'] = df['Year'].astype(int).astype(str)

In [333]:
df.groupby('Variable')['Value']

In [334]:
df = data[(data.Aggregation == 'Index')].groupby('Year').mean().reset_index()
fig = px.line(df,
              x='Year',
              y='Value',
              labels = {
                 'Value': ''
              },
             height=300)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(visible=True, fixedrange=True)
fig.update_traces(mode='lines+markers', line_color="#14ac9c")

In [382]:
df = data[(data.Aggregation.isin(['Dimension'])) & (data.Year.isin([2015, 2020]))].groupby(['Variable', 'Year']).mean().reset_index()
df['Value'] = df['Value'].round(1)
df['Year'] = df['Year'].astype(int).astype(str)
fig = px.scatter(df, x="Value", 
                 y="Variable",
                 color='Year',
                 labels={"Variable": 'Dimension'},
                 color_discrete_map={'2020': '#14ac9c', '2015':'darkgrey'},
)
fig.update_traces(marker=dict(size=25, opacity=0.6))
fig.update_xaxes(showgrid=False, range=[0,100])

fig

# current

In [227]:
df = data[data.Aggregation == 'Dimension'].groupby('Variable').mean().reset_index()

fig = px.scatter(df,
                x='Variable',
                y='Value',
                size=[8 for i in range(4)],
                color=['blue', 'blue', 'blue', 'blue'],
                 labels={
                     "Variable": "Dimension",
                 },
                )

fig.update_traces(marker=dict(size=12,
                              line=dict(width=12,
                                        color='#14ac9c')),
                  selector=dict(mode='markers'))


fig.add_trace(go.Bar(y=df['Value'], x=df['Variable'],
                     width=[5e-2 for i in range(4)],
                    marker_color=['#14ac9c', '#14ac9c', '#14ac9c', '#14ac9c']))

fig.update_layout(showlegend=False,hovermode=False)
fig.update_yaxes(range=[0, 100])

fig

In [226]:
df = data[data.Aggregation == 'Category'].groupby('Variable').mean().reset_index()

fig = go.Figure()

fig.add_trace(go.Scatterpolargl(
      r = df.Value,
      theta = df.Variable,
      name = "Trial 6",
      marker=dict(size=10, color="#14ac9c"),
      ))
fig.update_traces(fill='toself')
fig.update_traces(mode="markers", marker=dict(opacity=0.7))
fig.update_layout(margin={"r": 20, "t": 20, "l": 20, "b": 20})

## Trends

### World trend

In [84]:
df = data[(data.Aggregation == 'Index')].groupby('Year').mean().reset_index()
fig = px.line(df,
              x='Year',
              y='Value',
              labels = {
                 'Value': ''
              },
             height=300)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(visible=True, fixedrange=True)
fig.update_traces(mode='lines+markers', line_color="#14ac9c")

## Heatmaps

### Index

In [343]:
import plotly.express as px

df = data[(data.Aggregation == 'Index') & (data.Year == 2020)]
df = df.groupby(['Continent', 'Variable']).mean()['Value'].reset_index().pivot(index='Continent', columns='Variable')

fig = px.imshow(df['Value'],
          color_continuous_scale=[(0, "#fc8d59"), (0.5, "#ffffbf"), (1, "#14ac9c")],
          labels=dict(x="", y="Continent", color="Value"),
         range_color=[0, 100],
        width=300)


fig.update_layout(coloraxis_colorbar=dict(title="",
                                              thicknessmode="pixels", thickness=10,
                                              lenmode="pixels", len=100,
                                              dtick=50
                                              ))
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})


In [254]:
x = df['Value'].T.columns
x

Index(['Africa', 'America', 'Asia', 'Europe', 'Oceania'], dtype='object', name='Continent')

In [280]:
import plotly.figure_factory as ff

colorscale=[(0, "#fc8d59"), (0.5, "#ffffbf"), (1, "#14ac9c")]
font_colors = ['black']

x = list(df['Value'].T.columns.values)
y = ['Index']

fig = ff.create_annotated_heatmap(df['Value'].T.values.round(1),
                           colorscale=colorscale,
                           font_colors=font_colors,
                           x=x,
                           y=y,
                           hoverinfo='z',
                           )
fig.update_layout(height=300)
fig.layout.update(
  go.Layout(
    autosize=False,
    font=dict(
      family="roboto",
      size = 20)
    )
  )

fig

### Dimension

In [374]:
df = data[(data.Aggregation.isin(['Index', 'Dimension'])) & (data.Year == 2020)]
df = df.groupby(['Continent', 'Variable']).mean()['Value'].reset_index().pivot(index='Continent', columns='Variable')
df_plot = df['Value']
df_plot[''] = None

In [381]:
df = data[(data.Aggregation.isin(['Index', 'Dimension'])) & (data.Year == 2020)]
df = df.groupby(['Continent', 'Variable']).mean()['Value'].reset_index().pivot(index='Continent', columns='Variable')

fig = px.imshow(df_plot[['Index', '','ESRU', 'GEO', 'NCP', 'SI']],
          color_continuous_scale=[(0, "#fc8d59"), (0.6, "#ffffbf"), (1, "#14ac9c")],
          labels=dict(x="", y="", color="Value"),
          range_color=[0, 100],
          width=500
)

fig.update_layout(coloraxis_colorbar=dict(title="",
                                              thicknessmode="pixels", thickness=10,
                                              lenmode="pixels", len=200,
                                              dtick=20
                                              ))
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.layout.plot_bgcolor = '#fff'
fig

In [363]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1)
fig.append_trace(fig_dim, 1, 1)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Value: %{z}<extra></extra>',
              'type': 'heatmap',
              'x': array(['ESRU', 'GEO', 'NCP', 'SI'], dtype=object),
              'y': array(['Africa', 'America', 'Asia', 'Europe', 'Oceania'], dtype=object),
              'z': array([[42.98098039, 15.51041667, 62.14925926, 43.58433962],
                          [42.81774194, 22.4252381 , 64.96954545, 69.61909091],
                          [32.58108696, 23.64921053, 56.05479167, 66.38458333],
                          [54.68153846, 39.99131579, 70.09      , 84.8735    ],
                          [62.46833333, 17.905     , 62.63631579, 53.69166667]])}],
    'layout': {'coloraxis': {'cmax': 100,
                             'cmin': 0,
                             'colorbar': {'dtick': 20,
                        

In [344]:
import plotly.express as px

df = data[(data.Aggregation == 'Dimension') & (data.Year == 2020)]
df = df.groupby(['Continent', 'Variable']).mean()['Value'].reset_index().pivot(index='Continent', columns='Variable')

fig = px.imshow(df['Value'],
          color_continuous_scale=[(0, "#fc8d59"), (0.6, "#ffffbf"), (1, "#14ac9c")],
          labels=dict(x="Dimension", y="Continent", color="Value"),
               range_color=[0, 100],)


fig.update_layout(coloraxis_colorbar=dict(title="",
                                              thicknessmode="pixels", thickness=20,
                                              lenmode="pixels", len=200,
                                              dtick=20
                                              ))
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})


### Categories

In [213]:
import plotly.express as px

dims = data[(data.Aggregation == 'Category') & (data.Year == 2020)]
dims = dims.groupby(['Continent', 'Variable']).mean()['Value'].reset_index().pivot(index='Continent', columns='Variable')

fig = px.imshow(dims['Value'],
          color_continuous_scale=[(0, "#fc8d59"), (0.6, "#ffffbf"), (1, "#14ac9c")],
          labels=dict(x="Category", y="Continent", color="Value"),
               range_color=[0, 100],)


fig.update_layout(coloraxis_colorbar=dict(title="",
                                              thicknessmode="pixels", thickness=20,
                                              lenmode="pixels", len=200,
                                              dtick=20
                                              ))